In [ ]:
!pwd

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc

In [ ]:
# Define constants
IMAGE_WIDTH, IMAGE_HEIGHT = 150, 150
BATCH_SIZE = 32
EPOCHS = 5

In [ ]:

# Set paths to your dataset directories
TRAIN_DIR = './../dataset/image/set1/train'
TEST_DIR = './../dataset/image/set1/test'

In [ ]:
#Data preprocessing and augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(TRAIN_DIR,
                                                    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='binary')

test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                  target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='binary')

In [ ]:
# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=test_generator,
          validation_steps=test_generator.samples // BATCH_SIZE)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
# Function to visualize images with their predicted labels
def visualize_images(generator, model, num_images=5):
    # Get a batch of images and labels
    x_batch, y_batch = generator.next()
    # Make predictions
    predictions = model.predict(x_batch)
    
    # Plot the images with their predicted labels
    plt.figure(figsize=(15, 10))
    for i in range(num_images):
        plt.subplot(1, num_images, i+1)
        plt.imshow(x_batch[i])
        plt.title('Predicted: {:.2f}\nActual: {}'.format(predictions[i][0], y_batch[i]))
        plt.axis('off')
    plt.show()

# Visualize training images and predictions
print("Training Images and Predictions:")
visualize_images(train_generator, model)

# Visualize test images and predictions
print("Test Images and Predictions:")
visualize_images(test_generator, model)

In [ ]:
# Generate predictions for the test set
y_pred = model.predict(test_generator)
y_pred_classes = np.round(y_pred)

# Convert actual labels to binary (0 for real, 1 for fake)
y_true = test_generator.classes

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Real', 'Fake'])
plt.yticks(tick_marks, ['Real', 'Fake'])

# Add labels to each cell
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 ha="center", va="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true, y_pred.ravel())
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--', label='Random Guess')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Load and preprocess the new test data
new_test_data_dir = './../dataset/image/set2/test'

new_test_datagen = ImageDataGenerator(rescale=1./255)

new_test_generator = new_test_datagen.flow_from_directory(
    new_test_data_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False  # Important: Do not shuffle the data
)

# Generate predictions for the new test data
y_pred_new = model.predict(new_test_generator)

# Convert actual labels to binary (0 for real, 1 for fake)
y_true_new = new_test_generator.classes

# Calculate confusion matrix
cm_new = confusion_matrix(y_true_new, np.round(y_pred_new))

# Plot confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm_new, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix for New Test Data')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Real', 'Fake'])
plt.yticks(tick_marks, ['Real', 'Fake'])
thresh = cm_new.max() / 2.
for i in range(cm_new.shape[0]):
    for j in range(cm_new.shape[1]):
        plt.text(j, i, format(cm_new[i, j], 'd'),
                 ha="center", va="center",
                 color="white" if cm_new[i, j] > thresh else "black")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()

# Calculate ROC curve for new test data
fpr_new, tpr_new, thresholds_new = roc_curve(y_true_new, y_pred_new.ravel())
roc_auc_new = auc(fpr_new, tpr_new)

# Plot ROC curve for new test data
plt.figure(figsize=(8, 6))
plt.plot(fpr_new, tpr_new, color='blue', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc_new)
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--', label='Random Guess')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve for New Test Data')
plt.legend(loc='lower right')
plt.show()


In [ ]:
test_generator_2 = test_datagen.flow_from_directory("./../dataset/image/set2/test",
                                                  target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='binary')

# Visualize training images and predictions
print("Training Images and Predictions:")
visualize_images(test_generator_2, model)

# Visualize test images and predictions
print("Test Images and Predictions:")
visualize_images(test_generator_2, model)